In [61]:
%pylab inline
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor

Populating the interactive namespace from numpy and matplotlib


In [107]:
data = pd.read_csv('sample/train_set_weeks.csv', index_col='id')
categorical = ['idFilial', 'KanalDB', 'idSubGrp']

data = (data.merge(data.groupby(categorical, 
        as_index=False)['N wk'].max(), on=categorical))

test = data[data['N wk_x'] == data['N wk_y']]
data = data[data['N wk_x'] != data['N wk_y']]

test_labels = test['value']
test['value'] = None

data = data.append(test).reset_index(drop=True).sort_values('N wk_x')
data.index.name = 'id'

data['previous'] = data.groupby(categorical)['value'].shift(1)
data = data.dropna(subset=['previous']).sort_index()
grouped = data.groupby(categorical)['previous']

for j in range(2, 10):
    rolled = grouped.rolling(j)
    ravg = rolled.mean().reset_index()
    ravg = ravg.set_index('id').sort_index()
    data['avg{}'.format(j)] = ravg['previous']
    
for j in range(3, 10):
    rolled = grouped.rolling(j)
    rstd = rolled.std().reset_index()
    rstd = rstd.set_index('id').sort_index()
    data['std{}'.format(j)] = rstd['previous']
    
cumulative = (['avg{}'.format(j) for j in range(2, 10)] +
    ['std{}'.format(j) for j in range(3, 10)])
data = data.dropna(subset=cumulative).drop(['N wk_x', 'N wk_y', 'wk'], axis=1)

train_labels = data.loc[data['value'].notnull(), 'value']
test = data[data['value'].isnull()].drop('value', axis=1)
train = data[data['value'].notnull()].drop('value', axis=1)

In [108]:
encoder = OneHotEncoder(categorical_features=[0, 1, 2], sparse=False)
train = encoder.fit_transform(train)
test = encoder.transform(test)

In [109]:
regressor = Ridge(alpha=0.01, normalize=True)
regressor.fit(train, train_labels)
((regressor.predict(test) - test_labels)**2).mean()**0.5

45935.54974243528

In [ ]:
test_categorical['value'] = regressor_ridge.predict(test)
submit_ridge = (pd.read_csv('sample/test_set_weeks.csv',
    index_col='id').merge(test_categorical, on=categorical))
submit_ridge.index.name = 'id'
submit_ridge.to_csv('submit_ridge.csv')

In [4]:
model = LGBMRegressor(n_estimators=5000, boosting_type='gbdt', max_depth=8, learning_rate=0.05, logging_level='Verbose')
model.fit(train, train_labels, eval_set=(test, test_labels), eval_metric='rmse')

[1]	valid_0's rmse: 131858
[2]	valid_0's rmse: 131801
[3]	valid_0's rmse: 131743
[4]	valid_0's rmse: 131684
[5]	valid_0's rmse: 131627
[6]	valid_0's rmse: 131568
[7]	valid_0's rmse: 131511
[8]	valid_0's rmse: 131452
[9]	valid_0's rmse: 131394
[10]	valid_0's rmse: 131337
[11]	valid_0's rmse: 131280
[12]	valid_0's rmse: 131221
[13]	valid_0's rmse: 131165
[14]	valid_0's rmse: 131108
[15]	valid_0's rmse: 131051
[16]	valid_0's rmse: 130994
[17]	valid_0's rmse: 130937
[18]	valid_0's rmse: 130880
[19]	valid_0's rmse: 130823
[20]	valid_0's rmse: 130767
[21]	valid_0's rmse: 130710
[22]	valid_0's rmse: 130652
[23]	valid_0's rmse: 130594
[24]	valid_0's rmse: 130535
[25]	valid_0's rmse: 130477
[26]	valid_0's rmse: 130419
[27]	valid_0's rmse: 130361
[28]	valid_0's rmse: 130303
[29]	valid_0's rmse: 130245
[30]	valid_0's rmse: 130187
[31]	valid_0's rmse: 130129
[32]	valid_0's rmse: 130071
[33]	valid_0's rmse: 130013
[34]	valid_0's rmse: 129959
[35]	valid_0's rmse: 129905
[36]	valid_0's rmse: 129851
[

[292]	valid_0's rmse: 117275
[293]	valid_0's rmse: 117234
[294]	valid_0's rmse: 117194
[295]	valid_0's rmse: 117154
[296]	valid_0's rmse: 117114
[297]	valid_0's rmse: 117073
[298]	valid_0's rmse: 117032
[299]	valid_0's rmse: 116992
[300]	valid_0's rmse: 116948
[301]	valid_0's rmse: 116907
[302]	valid_0's rmse: 116865
[303]	valid_0's rmse: 116822
[304]	valid_0's rmse: 116779
[305]	valid_0's rmse: 116737
[306]	valid_0's rmse: 116694
[307]	valid_0's rmse: 116652
[308]	valid_0's rmse: 116609
[309]	valid_0's rmse: 116567
[310]	valid_0's rmse: 116522
[311]	valid_0's rmse: 116481
[312]	valid_0's rmse: 116439
[313]	valid_0's rmse: 116397
[314]	valid_0's rmse: 116355
[315]	valid_0's rmse: 116314
[316]	valid_0's rmse: 116272
[317]	valid_0's rmse: 116230
[318]	valid_0's rmse: 116189
[319]	valid_0's rmse: 116147
[320]	valid_0's rmse: 116106
[321]	valid_0's rmse: 116065
[322]	valid_0's rmse: 116022
[323]	valid_0's rmse: 115981
[324]	valid_0's rmse: 115940
[325]	valid_0's rmse: 115897
[326]	valid_0'

[627]	valid_0's rmse: 105345
[628]	valid_0's rmse: 105313
[629]	valid_0's rmse: 105281
[630]	valid_0's rmse: 105253
[631]	valid_0's rmse: 105221
[632]	valid_0's rmse: 105191
[633]	valid_0's rmse: 105161
[634]	valid_0's rmse: 105129
[635]	valid_0's rmse: 105101
[636]	valid_0's rmse: 105071
[637]	valid_0's rmse: 105040
[638]	valid_0's rmse: 105008
[639]	valid_0's rmse: 104977
[640]	valid_0's rmse: 104946
[641]	valid_0's rmse: 104918
[642]	valid_0's rmse: 104888
[643]	valid_0's rmse: 104857
[644]	valid_0's rmse: 104829
[645]	valid_0's rmse: 104797
[646]	valid_0's rmse: 104766
[647]	valid_0's rmse: 104735
[648]	valid_0's rmse: 104706
[649]	valid_0's rmse: 104677
[650]	valid_0's rmse: 104646
[651]	valid_0's rmse: 104615
[652]	valid_0's rmse: 104589
[653]	valid_0's rmse: 104558
[654]	valid_0's rmse: 104526
[655]	valid_0's rmse: 104500
[656]	valid_0's rmse: 104472
[657]	valid_0's rmse: 104446
[658]	valid_0's rmse: 104420
[659]	valid_0's rmse: 104392
[660]	valid_0's rmse: 104366
[661]	valid_0'

[971]	valid_0's rmse: 95969.6
[972]	valid_0's rmse: 95943.1
[973]	valid_0's rmse: 95916.6
[974]	valid_0's rmse: 95890.1
[975]	valid_0's rmse: 95863.6
[976]	valid_0's rmse: 95837.1
[977]	valid_0's rmse: 95810.6
[978]	valid_0's rmse: 95784.2
[979]	valid_0's rmse: 95757.7
[980]	valid_0's rmse: 95732.7
[981]	valid_0's rmse: 95706.2
[982]	valid_0's rmse: 95681.2
[983]	valid_0's rmse: 95654.7
[984]	valid_0's rmse: 95629.7
[985]	valid_0's rmse: 95603.3
[986]	valid_0's rmse: 95578.3
[987]	valid_0's rmse: 95551.9
[988]	valid_0's rmse: 95526.9
[989]	valid_0's rmse: 95500.5
[990]	valid_0's rmse: 95474.1
[991]	valid_0's rmse: 95449.1
[992]	valid_0's rmse: 95422.8
[993]	valid_0's rmse: 95397.8
[994]	valid_0's rmse: 95371.5
[995]	valid_0's rmse: 95345.2
[996]	valid_0's rmse: 95320.2
[997]	valid_0's rmse: 95293.9
[998]	valid_0's rmse: 95267.6
[999]	valid_0's rmse: 95242.7
[1000]	valid_0's rmse: 95216.4
[1001]	valid_0's rmse: 95191.5
[1002]	valid_0's rmse: 95165.2
[1003]	valid_0's rmse: 95140.4
[1004]

[1260]	valid_0's rmse: 89500.9
[1261]	valid_0's rmse: 89481.8
[1262]	valid_0's rmse: 89461.8
[1263]	valid_0's rmse: 89439.3
[1264]	valid_0's rmse: 89420.1
[1265]	valid_0's rmse: 89397.6
[1266]	valid_0's rmse: 89378.9
[1267]	valid_0's rmse: 89356.4
[1268]	valid_0's rmse: 89337.2
[1269]	valid_0's rmse: 89317.2
[1270]	valid_0's rmse: 89294.6
[1271]	valid_0's rmse: 89272.1
[1272]	valid_0's rmse: 89252.9
[1273]	valid_0's rmse: 89230.4
[1274]	valid_0's rmse: 89207.9
[1275]	valid_0's rmse: 89185.4
[1276]	valid_0's rmse: 89165.5
[1277]	valid_0's rmse: 89143
[1278]	valid_0's rmse: 89122.3
[1279]	valid_0's rmse: 89099.8
[1280]	valid_0's rmse: 89082.4
[1281]	valid_0's rmse: 89060
[1282]	valid_0's rmse: 89037.6
[1283]	valid_0's rmse: 89015.2
[1284]	valid_0's rmse: 88995.3
[1285]	valid_0's rmse: 88973
[1286]	valid_0's rmse: 88950.6
[1287]	valid_0's rmse: 88929.8
[1288]	valid_0's rmse: 88910
[1289]	valid_0's rmse: 88887.7
[1290]	valid_0's rmse: 88865.4
[1291]	valid_0's rmse: 88848
[1292]	valid_0's r

...

[3960]	valid_0's rmse: 49082.2
[3961]	valid_0's rmse: 49078.9
[3962]	valid_0's rmse: 49075.7
[3963]	valid_0's rmse: 49072.4
[3964]	valid_0's rmse: 49069.1
[3965]	valid_0's rmse: 49065.9
[3966]	valid_0's rmse: 49062.6
[3967]	valid_0's rmse: 49058.8
[3968]	valid_0's rmse: 49055.1
[3969]	valid_0's rmse: 49052
[3970]	valid_0's rmse: 49048.7
[3971]	valid_0's rmse: 49045.8
[3972]	valid_0's rmse: 49042.7
[3973]	valid_0's rmse: 49039.5
[3974]	valid_0's rmse: 49036.4
[3975]	valid_0's rmse: 49033.3
[3976]	valid_0's rmse: 49030.1
[3977]	valid_0's rmse: 49027
[3978]	valid_0's rmse: 49023.8
[3979]	valid_0's rmse: 49020.8
[3980]	valid_0's rmse: 49017.9
[3981]	valid_0's rmse: 49014.9
[3982]	valid_0's rmse: 49011.9
[3983]	valid_0's rmse: 49008.4
[3984]	valid_0's rmse: 49005.4
[3985]	valid_0's rmse: 49002.5
[3986]	valid_0's rmse: 48999.5
[3987]	valid_0's rmse: 48996.5
[3988]	valid_0's rmse: 48993.6
[3989]	valid_0's rmse: 48990.6
[3990]	valid_0's rmse: 48987.6
[3991]	valid_0's rmse: 48984.7
[3992]	valid

[4253]	valid_0's rmse: 47680.8
[4254]	valid_0's rmse: 47668.6
[4255]	valid_0's rmse: 47663.5
[4256]	valid_0's rmse: 47657.4
[4257]	valid_0's rmse: 47649.1
[4258]	valid_0's rmse: 47641.1
[4259]	valid_0's rmse: 47633.1
[4260]	valid_0's rmse: 47625.2
[4261]	valid_0's rmse: 47617.3
[4262]	valid_0's rmse: 47609.4
[4263]	valid_0's rmse: 47601.5
[4264]	valid_0's rmse: 47592.9
[4265]	valid_0's rmse: 47585.1
[4266]	valid_0's rmse: 47576.5
[4267]	valid_0's rmse: 47572.8
[4268]	valid_0's rmse: 47566.9
[4269]	valid_0's rmse: 47555.9
[4270]	valid_0's rmse: 47550.1
[4271]	valid_0's rmse: 47545
[4272]	valid_0's rmse: 47533.2
[4273]	valid_0's rmse: 47528.1
[4274]	valid_0's rmse: 47522.1
[4275]	valid_0's rmse: 47510
[4276]	valid_0's rmse: 47501.6
[4277]	valid_0's rmse: 47496.5
[4278]	valid_0's rmse: 47485.5
[4279]	valid_0's rmse: 47479.6
[4280]	valid_0's rmse: 47470.3
[4281]	valid_0's rmse: 47465.3
[4282]	valid_0's rmse: 47458.6
[4283]	valid_0's rmse: 47451.7
[4284]	valid_0's rmse: 47445.4
[4285]	valid

[4552]	valid_0's rmse: 45826.6
[4553]	valid_0's rmse: 45821.4
[4554]	valid_0's rmse: 45816.7
[4555]	valid_0's rmse: 45808.7
[4556]	valid_0's rmse: 45803.3
[4557]	valid_0's rmse: 45798.9
[4558]	valid_0's rmse: 45793.9
[4559]	valid_0's rmse: 45788.3
[4560]	valid_0's rmse: 45783.5
[4561]	valid_0's rmse: 45779
[4562]	valid_0's rmse: 45776.4
[4563]	valid_0's rmse: 45773.2
[4564]	valid_0's rmse: 45765.4
[4565]	valid_0's rmse: 45765.3
[4566]	valid_0's rmse: 45763.8
[4567]	valid_0's rmse: 45758.2
[4568]	valid_0's rmse: 45751.5
[4569]	valid_0's rmse: 45744.5
[4570]	valid_0's rmse: 45742.1
[4571]	valid_0's rmse: 45744
[4572]	valid_0's rmse: 45740.9
[4573]	valid_0's rmse: 45733.2
[4574]	valid_0's rmse: 45729.7
[4575]	valid_0's rmse: 45723
[4576]	valid_0's rmse: 45718.5
[4577]	valid_0's rmse: 45714.8
[4578]	valid_0's rmse: 45708.4
[4579]	valid_0's rmse: 45706.2
[4580]	valid_0's rmse: 45700.7
[4581]	valid_0's rmse: 45695.6
[4582]	valid_0's rmse: 45691.7
[4583]	valid_0's rmse: 45689.6
[4584]	valid_0

[4830]	valid_0's rmse: 44434.7
[4831]	valid_0's rmse: 44432.1
[4832]	valid_0's rmse: 44427
[4833]	valid_0's rmse: 44423.2
[4834]	valid_0's rmse: 44417.4
[4835]	valid_0's rmse: 44415.4
[4836]	valid_0's rmse: 44412.7
[4837]	valid_0's rmse: 44404.2
[4838]	valid_0's rmse: 44403.2
[4839]	valid_0's rmse: 44399
[4840]	valid_0's rmse: 44396.5
[4841]	valid_0's rmse: 44388.7
[4842]	valid_0's rmse: 44388.5
[4843]	valid_0's rmse: 44382.9
[4844]	valid_0's rmse: 44381.1
[4845]	valid_0's rmse: 44377.5
[4846]	valid_0's rmse: 44373.5
[4847]	valid_0's rmse: 44363.1
[4848]	valid_0's rmse: 44357.6
[4849]	valid_0's rmse: 44358.1
[4850]	valid_0's rmse: 44353.1
[4851]	valid_0's rmse: 44354.5
[4852]	valid_0's rmse: 44352.5
[4853]	valid_0's rmse: 44347.4
[4854]	valid_0's rmse: 44343.7
[4855]	valid_0's rmse: 44338.6
[4856]	valid_0's rmse: 44335.3
[4857]	valid_0's rmse: 44330.2
[4858]	valid_0's rmse: 44327.6
[4859]	valid_0's rmse: 44324.5
[4860]	valid_0's rmse: 44319.2
[4861]	valid_0's rmse: 44317.1
[4862]	valid

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.05,
       logging_level='Verbose', max_bin=255, max_depth=8,
       min_child_samples=10, min_child_weight=5, min_split_gain=0.0,
       n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
       random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=50000, subsample_freq=1)

In [3]:
test = pd.read_csv('sample/test_set_weeks.csv', index_col='id')
submit = test.merge(predict, on=categorical)
submit.index.name = 'id'
submit.to_csv('submit.csv')

In [ ]:
data = pd.read_csv('sample/train_set_weeks.csv', index_col='id')
data['N wk'] **= 8
data = data.merge(data.groupby(categorical)['N wk'].sum().
    rename('normalization').reset_index(), on=categorical)
data['N wk'] /= data['normalization']
data['value'] *= data['N wk']
predict = data.groupby(categorical)['value'].sum().reset_index()